In [ ]:
import time
from datetime import datetime, timedelta
import yfinance as yf
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def get_usd_to_krw_yf(max_lookback=7):
    symbol = "USDKRW=X"
    today = datetime.today()
    for i in range(1, max_lookback + 1):
        day = today - timedelta(days=i)
        start = day.strftime('%Y-%m-%d')
        end = (day + timedelta(days=1)).strftime('%Y-%m-%d')
        data = yf.Ticker(symbol).history(start=start, end=end)
        if not data.empty:
            return int(float(data['Close'].iloc[0]))
    return None

def get_previous_trading_day(ticker_symbol, max_lookback=7):
    today = datetime.today()
    for i in range(1, max_lookback + 1):
        day = today - timedelta(days=i)
        data = yf.Ticker(ticker_symbol).history(
            start=day.strftime('%Y-%m-%d'),
            end=(day + timedelta(days=1)).strftime('%Y-%m-%d')
        )
        if not data.empty:
            return data['Close'].iloc[0]
    return None

def calculate_feed_price_per_kg(corn_price, exchange_rate):
    return round((corn_price / 100 / 27.2 * exchange_rate), 2)

def get_avg_pig_price_kr_per_kg():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
        today = time.strftime("%Y-%m-%d")
        url = f"https://www.ekapepia.com/supPrice/auction/price/pigNew.do?menuSn=65&searchStartDate={today}&searchEndDate={today}&pageType=main&gubunType=#"
        driver.get(url)
        time.sleep(3)

        selector = "#content > div.sub-in > section:nth-child(10) > div > div.table-basic.ofx > table > tbody > tr:nth-child(7) > td:nth-child(19)"
        element = driver.find_element(By.CSS_SELECTOR, selector)
        raw_text = element.text.strip()
        cleaned_text = raw_text.split("\n")[0].replace(",", "")
        return float(cleaned_text)

    except Exception as e:
        print("경락가 가져오기 실패:", e)
        return None
    finally:
        driver.quit()

def calculate_pig_farm_investment(
    num_pigs,
    feed_price,
    avg_feed_kg,
    cost_building,
    water_electricity_cost,
    disease_control_cost,
    vehicle_cost,
    baby_pig_cost,
    labor_cost,
    manure_disposal_cost,
    general_management_cost,
    land_cost,
    other_costs
):
    cost_feed = feed_price * avg_feed_kg
    total_cost = (
        cost_feed + cost_building + water_electricity_cost +
        disease_control_cost + vehicle_cost + baby_pig_cost + labor_cost +
        manure_disposal_cost + general_management_cost + land_cost + other_costs
    ) * num_pigs
    return total_cost

def main():
    rate = get_usd_to_krw_yf()
    corn_price = get_previous_trading_day("ZC=F")

    if not (rate and corn_price):
        print("환율정보 불러오기 실패")
        return

    feed_price = calculate_feed_price_per_kg(corn_price, rate)
    print(f"현재 계산된 사료 단가: {feed_price:.2f}원/kg")

    num_pigs = int(input("사육할 돼지 마리수를 입력하세요: "))
    avg_weight_kg = float(input("평균 돼지 몸무게를 입력하세요 (kg): "))
    avg_feed_kg = float(input("출하 전까지 먹은 사료 총량 (kg): "))

    total_cost = calculate_pig_farm_investment(
        num_pigs=num_pigs,
        feed_price=feed_price,
        avg_feed_kg=avg_feed_kg,
        cost_building=9644,
        water_electricity_cost=7533,
        disease_control_cost=10637,
        vehicle_cost=1798,
        baby_pig_cost=82075,
        labor_cost=25482,
        manure_disposal_cost=12405,
        general_management_cost=2155,
        land_cost=16779,
        other_costs=16879
    )

    avg_market_price = get_avg_pig_price_kr_per_kg()
    if avg_market_price is None:
        print("경락가 없음")
        return

    total_revenue = avg_weight_kg * avg_market_price * num_pigs
    profit = total_revenue - total_cost

    print(f"총 투자 비용: {total_cost:,.0f}원")
    print(f"총 예상 수익: {total_revenue:,.0f}원")
    print(f"순수익(예상): {profit:,.0f}원")
if __name__ == "__main__":
    main()


현재 계산된 사료 단가: 246.71원/kg
총 투자 비용: 28,979,850원
총 예상 수익: 65,331,500원
순수익(예상): 36,351,650원
